In [ ]:
# import dependencies
import shutil
import os
from glob import glob
import re
import mlflow.azureml
from mlflow.keras import load_model
from azureml.core import Workspace
from azureml.core.webservice import Webservice
from azureml.core.authentication import ServicePrincipalAuthentication

In [ ]:
# please insert the path to model archive
model_zip_path = '<insert path to model here>'

In [ ]:
# get all enviroment variables
account_name = os.getenv("account_name")
account_key = os.getenv("account_key")
workspace_name = os.getenv('workspace_name')
subscription_id = os.getenv('subscription_id')
resource_group = os.getenv('resource_group')
location = os.getenv('location')
tenant_id = os.getenv('tenant_id')
service_principal_id = os.getenv('service_principal_id')
service_principal_password = os.getenv('service_principal_password')

In [ ]:
model_local_dir = 'model'

In [ ]:
# unpack the model
shutil.unpack_archive(model_zip_path, model_local_dir)

In [ ]:
# test if model is loaded properly
load_model(model_local_dir)

In [ ]:
# rename as not all symbols are allowed by azure
fileName = model_zip_path[model_zip_path.find('/')+1:-4]
fileName = re.sub(r'[^-\.a-z0-9]', '-', fileName.lower())
image_name = re.sub(r'\W+', '', fileName) + '-image'
model_name  = re.sub(r'\W+', '', fileName) + '-model'
deploy_name = re.sub(r'\W+', '', fileName) + '-deploy'

In [ ]:
# create authorization object
auth = ServicePrincipalAuthentication(
    tenant_id=tenant_id,
    service_principal_id=service_principal_id,
    service_principal_password= service_principal_password)

In [ ]:
# create/get workspace
azure_workspace = Workspace.create(
    name=workspace_name,
    subscription_id=subscription_id,
    resource_group=resource_group,
    location=location,
    create_resource_group=True,
    exist_ok=True,
    auth = auth)

In [ ]:
# build the image and save in azure container registry
azure_image, azure_model = mlflow.azureml.build_image(
    model_uri=model_local_dir,
    workspace=azure_workspace,
    synchronous=True,
    image_name = image_name,
    model_name = model_name)

In [ ]:
# deploy container instance from the image
webservice = Webservice.deploy_from_image(
    image = azure_image, 
    workspace=azure_workspace, 
    name=deploy_name)
webservice.wait_for_deployment()